In [ ]:
# download nc files based on URL and patch together
# https://www.ngdc.noaa.gov/thredds/catalog/global/ETOPO2022/15s/15s_geoid_netcdf/catalog.html?dataset=globalDatasetScan/ETOPO2022/15s/15s_geoid_netcdf/ETOPO_2022_v1_15s_N00E000_geoid.nc#:~:text=the%20DAP2%20protcol.-,HTTPServer,-Data%20Access

In [76]:
import requests
from bs4 import BeautifulSoup
from pathlib import Path
import urllib
import xarray as xa

In [88]:
def get_n_last_subparts_path(dir_path: Path | str, n: int) -> Path:
	"""Returns "n" last parts of a path. E.g. /first/second/third/fourth with n = 3 will return second/third/fourth"""
	return Path(*Path(dir_path).parts[-n:])


def check_path_suffix(path: Path | str, comparison: str) -> bool:
	"""Checks whether path provided ends in a particular suffix e.g. "nc". Since usually forget to specify ".", first
	tries to pad "comparison" with a period. TODO: docstring"""

	# pad with leading "."
	if "." not in comparison:
		comparison = "." + comparison

	if Path(path).suffix == comparison:
		return True
	else:
		return False


def download_etopo_data(download_dest_dir: Path | str, resolution: str | int) -> None:
	"""Download data"""
	bathymetry_url_page = f"https://www.ngdc.noaa.gov/thredds/catalog/global/ETOPO2022/{resolution}s/{resolution}s_geoid_netcdf/catalog.html"
	file_server_url = "https://www.ngdc.noaa.gov/thredds/fileServer/global/"

	reqs = requests.get(bathymetry_url_page)
	soup = BeautifulSoup(reqs.text, 'html.parser')

	# traverse paragraphs from soup TODO: add progress bar
	for link in soup.find_all("a"):
		# if ends with nc
		if check_path_suffix(link.get("href"), "nc"):
			file_specifier = get_n_last_subparts_path(Path(link.get("href")), 4)
			file_name = get_n_last_subparts_path(Path(link.get("href")), 1)

			# download file TODO: put this into a function (including checking if already exists)
			r = requests.get(file_server_url + str(file_specifier))
			with open(Path(download_dest_dir, file_name), 'wb') as f:
				f.write(r.content)
			# print(file_specifier)


def load_merge_nc_files(nc_dir: Path | str) -> xa.DataSet:
	files = Path(nc_dir).glob("*.nc")
	# combine nc files by coordinates
	merged_ncs = xa.open_mfdataset(files)


In [89]:
download_etopo_data('/Users/orlandotimmerman/Library/CloudStorage/OneDrive-UniversityofCambridge/cambridge/mres/mres_project/coralshift/datasets/bathymetry/ETOPO22', 15)

KeyboardInterrupt: 

In [ ]:
url = "https://www.ngdc.noaa.gov/thredds/fileServer/global/ETOPO2022/15s/15s_geoid_netcdf/ETOPO_2022_v1_15s_N00E000_geoid.nc"

# download file
r = requests.get(file_server_url, file_specifier)
with open(Path(download_dest_dir, file_name), 'wb') as f:
	f.write(r.content)

In [68]:
bathymetry_url_page = "https://www.ngdc.noaa.gov/thredds/catalog/global/ETOPO2022/15s/15s_geoid_netcdf/catalog.html"
reqs = requests.get(bathymetry_url_page)
soup = BeautifulSoup(reqs.text, 'html.parser')


url_start = "https://www.ngdc.noaa.gov/thredds/catalog/global/ETOPO2022/15s/15s_geoid_netcdf/"
access_str = "#:~:text=the%20DAP2%20protcol.-,HTTPServer,-Data%20Access"

# traverse paragraphs from soup
for link in soup.find_all("a"):
    # if ends with nc
    if check_path_suffix(link.get("href"), "nc"):
        file_specifier = get_n_last_subparts_path(Path(link.get("href")), 3)
        # download file
        print(file_specifier)

#https://www.ngdc.noaa.gov/thredds/fileServer/global/ETOPO2022/15s/15s_geoid_netcdf/ETOPO_2022_v1_15s_N00E000_geoid.nc
   # https://www.ngdc.noaa.gov/thredds/catalog/global/ETOPO2022/15s/15s_geoid_netcdf/catalog.html?dataset=globalDatasetScan/ETOPO2022/15s/15s_geoid_netcdf/ETOPO_2022_v1_15s_N00E000_geoid.nc#:~:text=the%20DAP2%20protcol.-,HTTPServer,-Data%20Access

15s/15s_geoid_netcdf/ETOPO_2022_v1_15s_N00E000_geoid.nc
15s/15s_geoid_netcdf/ETOPO_2022_v1_15s_N00E015_geoid.nc
15s/15s_geoid_netcdf/ETOPO_2022_v1_15s_N00E030_geoid.nc
15s/15s_geoid_netcdf/ETOPO_2022_v1_15s_N00E045_geoid.nc
15s/15s_geoid_netcdf/ETOPO_2022_v1_15s_N00E060_geoid.nc
15s/15s_geoid_netcdf/ETOPO_2022_v1_15s_N00E075_geoid.nc
15s/15s_geoid_netcdf/ETOPO_2022_v1_15s_N00E090_geoid.nc
15s/15s_geoid_netcdf/ETOPO_2022_v1_15s_N00E105_geoid.nc
15s/15s_geoid_netcdf/ETOPO_2022_v1_15s_N00E120_geoid.nc
15s/15s_geoid_netcdf/ETOPO_2022_v1_15s_N00E135_geoid.nc
15s/15s_geoid_netcdf/ETOPO_2022_v1_15s_N00E150_geoid.nc
15s/15s_geoid_netcdf/ETOPO_2022_v1_15s_N00E165_geoid.nc
15s/15s_geoid_netcdf/ETOPO_2022_v1_15s_N00W015_geoid.nc
15s/15s_geoid_netcdf/ETOPO_2022_v1_15s_N00W030_geoid.nc
15s/15s_geoid_netcdf/ETOPO_2022_v1_15s_N00W045_geoid.nc
15s/15s_geoid_netcdf/ETOPO_2022_v1_15s_N00W060_geoid.nc
15s/15s_geoid_netcdf/ETOPO_2022_v1_15s_N00W075_geoid.nc
15s/15s_geoid_netcdf/ETOPO_2022_v1_15s_N00W090_g

None


In [9]:

url = 'https://www.geeksforgeeks.org/'
reqs = requests.get(url)
soup = BeautifulSoup(reqs.text, 'html.parser')

urls = []
for link in soup.find_all('a'):
	print(link.get('href'))


#main
https://www.geeksforgeeks.org/
https://practice.geeksforgeeks.org/courses/geeks-classes-live?utm_source=gfg&utm_medium=Submenu&utm_campaign=courses-submenu
https://practice.geeksforgeeks.org/courses/system-design-live?utm_source=gfg&utm_medium=Submenu&utm_campaign=courses-submenu
https://practice.geeksforgeeks.org/courses/devops-live?utm_source=gfg&utm_medium=Submenu&utm_campaign=courses-submenu
https://practice.geeksforgeeks.org/courses/category/all?courseType=live&utm_source=gfg&utm_medium=Submenu&utm_campaign=courses-submenu
https://practice.geeksforgeeks.org/courses/complete-interview-preparation?utm_source=gfg&utm_medium=Submenu&utm_campaign=courses-submenu
https://practice.geeksforgeeks.org/courses/data-science-live?utm_source=gfg&utm_medium=Submenu&utm_campaign=courses-submenu
https://practice.geeksforgeeks.org/courses/gate-live-course/?utm_source=geeksforgeeks&utm_medium=submenu_22&utm_campaign=courses-submenu
https://practice.geeksforgeeks.org/courses/dsa-self-paced?utm_